In [ ]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [ ]:
cars = []
url = "https://kolesa.kz/cars/?year[from]=1990&year[to]=2017&price[from]=15%20000%20000&page={}"
url = "https://kolesa.kz/cars/"
import time

print(url.format('123'))

In [ ]:
attr_to_index = {'Марка':0, 
                 'Год':1, 
                 'Цена':2, 
                 'Город':3, 
                 'Кузов':4,
                 'Объем двигателя, л':5,
                 'Пробег':6,
                 'Коробка передач':7,
                 'Руль':8,
                 'Цвет':9,
                 'Привод':10,
                 'Растаможен':11,
                 'VIN':12,
                 'Бензин':13}
for i in range(1, 500):
    source = urllib.request.urlopen(url.format(i+1)).read()
    soup = BeautifulSoup(source)   
    #print(soup)
#     break
    #print(soup.title)
    divs = soup.findAll('div', {'class': 'a-info-side col-right-list'})
    for div in divs:
        
        line = [None for _ in range(14)]
        line[0] = div.find('a', {'class': 'list-link ddl_product_link'}).string
#         line.append(div.find('div', {'class': 'list-region'}).string.strip())
        line[1] = div.find('div', {'class': 'a-search-description'}).string.strip()[:4]
        line[2] = str(div.find('span', {'class': 'price'}).text).strip()[:-2].replace(u'\xa0', u'')
        href = div.find('a', href=True)["href"]
        car_url = "https://kolesa.kz{}".format(href)
        car_source = urllib.request.urlopen(car_url).read()
        car_soup = BeautifulSoup(car_source)
        car_attr = car_soup.findAll('div', {'class': 'offer__parameters'})[0]
        car_attr = car_attr.findAll('dl')
#         print(car_attr)
        for attr in car_attr:
            attr_type = attr.find('dt')
#             print(attr_type)
            if attr_type.string == "Наличие":
                continue
            try:
                attr_type = attr_type['title']
            except:
                continue
            
            if (attr_type == "Объем двигателя, л"):
                engine_volume = attr.find('dd').string.split()[0]
                is_petroleum = attr.find('dd').string.split()[1]
                line[attr_to_index[attr_type]] = engine_volume
                line[attr_to_index["Бензин"]] = is_petroleum[1:-1]
            elif (attr_type == "Пробег"):
                attr_str = attr.find('dd').string.strip()[:-3]
                val_str = "".join(attr_str.split())
                line[attr_to_index[attr_type]] = int(val_str)
            else:
                attr_value = attr.find('dd').string.strip()
                try:
                    line[attr_to_index[attr_type]] = attr_value
                except:
                    continue
            
        #print(line)
        cars.append(line)



import csv
with open("kolesa_cars.csv", 'w', newline='\n') as myfile:
    wr = csv.writer(myfile)
    for c in cars:
        wr.writerow(c)

In [ ]:
print(cars[0:2])

In [ ]:
cars_with_probeg = []
for c in cars:
    if (c[6] != None):
        cars_with_probeg.append(c)

In [ ]:
len(cars_with_probeg)

In [ ]:
import copy

cars_copy = copy.copy(cars_with_probeg)

In [ ]:
cars = cars_with_probeg

import pandas as pd
import numpy as np
import math

attr_to_index = {'Марка':0, 
                 'Год':1, 
                 'Цена':2, 
                 'Город':3, 
                 'Кузов':4,
                 'Объем двигателя, л':5,
                 'Пробег':6,
                 'Коробка передач':7,
                 'Руль':8,
                 'Цвет':9,
                 'Привод':10,
                 'Растаможен':11,
                 'VIN':12,
                 'Бензин':13}

X = None
for idx in [0, 3, 4, 7, 8, 9, 11, 13]:    
    categorical_feats = [c[idx] for c in cars]
#     for f in categorical_feats:
#         if (f == None):
#             print("omitting feature {}".format(idx))
    unique_feats = set(categorical_feats)
    print(len(unique_feats))
#     if idx in [7,8,9,10,11,13]:
    print(unique_feats)
    s = pd.Series(categorical_feats)
    dummies = pd.get_dummies(s)
    if X is None:
        X = dummies.values
    else:
        X = np.append(X, dummies.values, axis=1)
    print("X shape = {}".format(X.shape))
    print("dummies shape = {}".format(dummies.values.shape))
    
for idx in [1, 5, 6]:
    numeric_feats = [c[idx] for c in cars]
    numeric_feats = np.expand_dims(np.array(numeric_feats), axis=1)
    print("X shape = {}".format(X.shape))
    print("features shape = {}".format(numeric_feats.shape))
    X = np.append(X, numeric_feats, axis=1)

y = np.array([c[2] for c in cars])


In [ ]:
dataset_size = X.shape[0]
train_size = math.floor(dataset_size * 0.8)
test_size = dataset_size - train_size
print("train_size = {}".format(train_size))
print("test_size = {}".format(test_size))
print("dataset_size = {}".format(dataset_size))

X_train = X[:train_size].astype(np.float32)
y_train = y[:train_size].astype(np.float32)

X_test = X[train_size:].astype(np.float32)
y_test = y[train_size:].astype(np.float32)

print("X_train = {}".format(X_train.shape))

#Methods


In [ ]:
import random
import copy

class Lasso():
    
    def __init__(self, num_weights, lambd, num_iters=1000):
        self.num_weights = num_weights
        self.w = np.ones((num_weights,)).astype(np.float)
        self.lambd = lambd
        self.is_converged = False
        self.num_iters = num_iters
    def fit(self, X, y):
        weights = []
        for _ in range(self.num_iters):
            i = random.choice(range(self.num_weights))
            r = -y
            for j in range(X.shape[1]):
                if j != i:
                    r += np.dot(X[:,j], self.w[j])
            a = -2 * np.dot(X[:,i], r)
            b = 2 * np.sum(np.square(X[:,i]))
            if abs(a) <= self.lambd:
                self.w[i] = 0.0
            else: 
                det = (-self.lambd + a) / b
                if det > 0:
                    self.w[i] = det
                else:
                    self.w[i] = (self.lambd + a) / b
            if len(weights) == 3:
#                 print(self.w)
                prev_mean = np.array(weights).mean()
                if np.linalg.norm(prev_mean - self.w) < 0.1:
                    break
                w_copy = copy.copy(weights)
                weights = []
                for w_i in range(2):
                    weights.append(w_copy[w_i+1])
                weights.append(self.w)
            else:
                weights.append(self.w)
        
    def predict(self, X):
        return np.dot(X, self.w)

In [ ]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor


reg = LinearRegression().fit(X_train, y_train)
print(reg.score(X_train, y_train))
print(reg.score(X_test, y_test))

y_pred = reg.predict(X_test)

print("Mean error = {}".format(math.sqrt(sklearn.metrics.mean_squared_error(y_pred, y_test))))

lasso_model = Lasso(X_train.shape[1], 0.01)
lasso_model.fit(X_train, y_train)
y_pred = lasso_model.predict(X_test)
print("Mean error = {}".format(math.sqrt(sklearn.metrics.mean_squared_error(y_pred, y_test))))

scikit_lasso = sklearn.linear_model.Lasso()
scikit_lasso.fit(X_train, y_train)
y_pred = scikit_lasso.predict(X_test)
print("Mean error = {}".format(math.sqrt(sklearn.metrics.mean_squared_error(y_pred, y_test))))

scikit_ridge = sklearn.linear_model.Ridge()
scikit_ridge.fit(X_train, y_train)
y_pred = scikit_ridge.predict(X_test)
print("Mean error = {}".format(math.sqrt(sklearn.metrics.mean_squared_error(y_pred, y_test))))

nn = MLPRegressor((100, 100, 100))
nn.fit(X_train, y_train)
y_pred = nn.predict(X_test)
print("Mean error = {}".format(math.sqrt(sklearn.metrics.mean_squared_error(y_pred, y_test))))